# Chapter1 : GETTING TO KNOW PYSPARK

## What is spark?
Spark is a platform for cluster computing. Spark lets you spread data and computations over clusters with multiple nodes (think of each node as a separate computer). Splitting up your data makes it easier to work with very large datasets because each node only works with a small amount of data.

## Using Spark in Python
In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the master that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called worker. The master sends the workers data and calculations to run, and they send their results back to the master.

## Using Dataframes
Spark's core data structure is the Resilient Distributed Dataset (RDD). This is a low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster. However, RDDs are hard to work with directly, so in this course you'll be using the Spark DataFrame abstraction built on top of RDDs.
The Spark DataFrame was designed to behave a lot like a SQL table (a table with variables in the columns and observations in the rows). Not only are they easier to understand, DataFrames are also more optimized for complicated operations than RDDs.
When you start modifying and combining columns and rows of data, there are many ways to arrive at the same result, but some often take much longer than others. When using RDDs, it's up to the data scientist to figure out the right way to optimize the query, but the DataFrame implementation has much of this optimization built in!
To start working with Spark DataFrames, you first have to create a SparkSession object from your SparkContext. You can think of the SparkContext as your connection to the cluster and the SparkSession as your interface with that connection.


In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\SPARK\\spark-2.4.5-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [3]:
print(sc)

print(sc.version)

<SparkContext master=local[*] appName=pyspark-shell>
2.4.5


## EXERCISE 

### 1:Creating a SparkSession
Import SparkSession from pyspark.sql.

Make a new SparkSession called spark using SparkSession.builder.getOrCreate().

Print spark to the console to verify it's a SparkSession.

In [3]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
print(spark)

### 2:VIEWING TABLES

Once you've created a SparkSession, you can start poking around to see what data is in your cluster!

Your SparkSession has an attribute called catalog which lists all the data inside the cluster. This attribute has a few methods for extracting different pieces of information.

One of the most useful is the .listTables() method, which returns the names of all the tables in your cluster as a list.

### INSTRUCTION
1: See what tables are in your cluster by calling spark.catalog.listTables() and printing the result!

In [4]:
files = "E:/STUDY/flights_small.csv"

# Read in the flights data
flights_temp = spark.read.csv(files, header=True)
print(flights_temp)

# Add flights_temp to the catalog
flights_temp.createOrReplaceTempView('flights')

#Examine the tables in the catalog
print(spark.catalog.listTables())

flights_temp.show(113) #showwing whole data 

DataFrame[year: string, month: string, day: string, dep_time: string, dep_delay: string, arr_time: string, arr_delay: string, carrier: string, tailnum: string, flight: string, origin: string, dest: string, air_time: string, distance: string, hour: string, minute: string]
[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|     

## 3:ARE YOU QUER-IOUS?
1: Use the .sql() method to get the first 10 rows of the flights table and save the result to flights10. The variable query contains the appropriate SQL query.

2: Use the DataFrame method .show() to print flights10

In [5]:
# Don't change this query
query = "FROM flights SELECT * LIMIT 10"

# Get the first 10 rows of flights
flights10 = spark.sql(query)

# Show the results
flights10.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

### 4:PANDAFY A SPARK DATAFRAME 

Suppose you've run a query on your huge dataset and aggregated it down to something a little more manageable.

Sometimes it makes sense to then take that table and work with it locally using a tool like **pandas**. Spark DataFrames make that easy with the **.toPandas()** method. Calling this method on a Spark DataFrame returns the corresponding pandas DataFrame. It's as simple as that!

This time the query counts the number of flights to each airport from SEA and PDX.

### INSRUCTIONS
1: Run the query using the .sql() method. Save the result in flight_counts.

2: Use the .toPandas() method on flight_counts to create a pandas DataFrame called pd_counts.
3: Print the .head() of pd_counts to the console.

In [26]:
# Don't change this query
query = "SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest"

#query = "SELECT origin COUNT(*) as N FROM flights GROUP BY origin"
# Run the query
flight_counts = spark.sql(query)

# Convert the results to a pandas DataFrame
pd_counts = flight_counts.toPandas()

print(pd_counts)

# Print the head of pd_counts
print(pd_counts.head()) #look sql in a nutshell to understand ths cell

    origin dest    N
0      SEA  RNO    8
1      SEA  DTW   98
2      SEA  CLE    2
3      SEA  LAX  450
4      PDX  SEA  144
..     ...  ...  ...
108    SEA  JFK  127
109    SEA  FAI   72
110    PDX  HOU    6
111    PDX  BUR   71
112    SEA  MCO   31

[113 rows x 3 columns]
  origin dest    N
0    SEA  RNO    8
1    SEA  DTW   98
2    SEA  CLE    2
3    SEA  LAX  450
4    PDX  SEA  144


## PUT SOME SPARK IN YOUR DATA
In the last exercise, you saw how to move data from Spark to pandas. However, maybe you want to go the other direction, and put a pandas DataFrame into a Spark cluster! The SparkSession class has a method for this as well.

The .createDataFrame() method takes a pandas DataFrame and returns a Spark DataFrame.

The output of this method is stored locally, not in the SparkSession catalog. This means that you can use all the Spark DataFrame methods on it, but you can't access the data in other contexts.

For example, a SQL query (using the .sql() method) that references your DataFrame will throw an error. To access the data in this way, you have to save it as a temporary table.

You can do this using the .createTempView() Spark DataFrame method, which takes as its only argument the name of the temporary table you'd like to register. This method registers the DataFrame as a table in the catalog, but as this table is temporary, it can only be accessed from the specific SparkSession used to create the Spark DataFrame.

There is also the method .createOrReplaceTempView(). This safely creates a new temporary table if nothing was there before, or updates an existing table if one was already defined. You'll use this method to avoid running into problems with duplicate tables.

### INSTRUCTIONS

1: The code to create a pandas DataFrame of random numbers has already been provided and saved under pd_temp.

2: Create a Spark DataFrame called spark_temp by calling the .createDataFrame() method with pd_temp as the argument.

3: Examine the list of tables in your Spark cluster and verify that the new DataFrame is not present. Remember you can use spark.catalog.listTables() to do so.

4: Register spark_temp as a temporary table named "temp" using the .createOrReplaceTempView() method. Remember that the table name is set including it as the only argument!

5: Examine the list of tables again! 

In [6]:
import pandas as pd
import numpy as np

# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))
print(pd_temp)

# Create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)  #.createDataFrame() takes a pandas DataFrame and return a spark DataFrame

# Examine the tables in the catalog
print(spark.catalog.listTables())

# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView('temp')

spark_temp.show()

          0
0  0.724446
1  0.480262
2  0.995138
3  0.350692
4  0.578659
5  0.182186
6  0.368969
7  0.696586
8  0.934504
9  0.305591
[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
+-------------------+
|                  0|
+-------------------+
| 0.7244462434825681|
| 0.4802617042454008|
| 0.9951378487604144|
| 0.3506915053772429|
| 0.5786587636112868|
|0.18218645435026615|
|0.36896855906185044|
| 0.6965855569602576|
| 0.9345044784496342|
| 0.3055911817921111|
+-------------------+



### 5:DROPPING THE MIDDLE MAN 
1: Use the .read.csv() method to create a Spark DataFrame called airports

2: The first argument is file_path

3: Pass the argument header=True so that Spark knows to take the column names from the first line of the file.

4: Print out this DataFrame by calling .show() 

In [22]:
# Don't change this file path
file_path = "E:/STUDY/airports.csv"

# Read in the airports data
airports = spark.read.csv(file_path, header=True)

# Show the data
airports.show()

+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

# Chapter:2 MANIPULATING DATA
In this chapter, you'll learn how to use the methods defined by Spark's DataFrame class to perform common data operations.

Let's look at performing column-wise operations. In Spark you can do this using the **.withColumn()** method, which takes two arguments. First, a string with the name of your new column, and second the new column itself.

The new column must be an object of class **Column**. Creating one of these is as easy as extracting a column from your DataFrame using **df.colName**.

Updating a Spark DataFrame is somewhat different than working in **pandas** because the Spark DataFrame is immutable. This means that it can't be changed, and so columns can't be updated in place.

Thus, all these methods return a new DataFrame. To overwrite the original DataFrame you must reassign the returned DataFrame using the method like so:

            df = df.withColumn("newCol", df.oldCol + 1)

The above code creates a DataFrame with the same columns as **df** plus a new column, **newCol**, where every entry is equal to the corresponding entry from **oldCol**, plus one.

To overwrite an existing column, just pass the name of the column as the first argument!

## EXERCISE 

### 1:CREATING COLUMN
1: Use the spark.table() method with the argument "flights" to create a DataFrame containing the values of the flights table in the .catalog. Save it as flights.

2: Show the head of flights using flights.show(). The column air_time contains the duration of the flight in minutes.

3: Update flights to include a new column called duration_hrs, that contains the duration of each flight in hours.

In [7]:
# Create the DataFrame flights
flights = spark.table('flights')

# Show the head
flights.show()

# Add duration_hrs
flights = flights.withColumn('duration_hrs', flights.air_time/60)
flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

## SQL IN A NUTSHELL
Another common database task is aggregation. That is, reducing your data by breaking it into chunks and summarizing each chunk.

This is done in SQL using the **GROUP BY** command. This command breaks your data into groups and applies a function from your **SELECT** statement to each group.

For example, if you wanted to count the number of flights from each of two origin destinations, you could use the query:
     
     SELECT COUNT(*) FROM flights GROUP BY origin;
     
**GROUP BY origin** tells SQL that you want the output to have a row for each unique value of the **origin** column. The **SELECT** statement selects the values you want to populate each of the columns. Here, we want to **COUNT()** every row in each of the groups.

It's possible to **GROUP BY** more than one column. When you do this, the resulting table has a row for every combination of the unique values in each column. The following query counts the number of flights from SEA and PDX to every destination airport:
      
      SELECT origin, dest, COUNT(*) FROM flights GROUP BY origin, dest;

The output will have a row for every combination of the values in **origin** and **dest** (i.e. a row listing each origin and destination that a flight flew to). There will also be a column with the **COUNT()** of all the rows in each group.

## 2:FILTERING DATA

For example, the following two expressions will produce the same output:

  **flights.filter("air_time > 120").show()** 

  **flights.filter(flights.air_time > 120).show()**

Notice that in the first case, we pass a string to .filter(). In SQL, we would write this filtering task as **SELECT * FROM flights WHERE air_time > 120**. Spark's **.filter()** can accept any expression that could go in the **WHERE** clause of a SQL query (in this case, **"air_time > 120"**), as long as it is passed as a string. Notice that in this case, we do not reference the name of the table in the string -- as we wouldn't in the SQL request.

In the second case, we actually pass a column of boolean values to **.filter()**. Remember that **flights.air_time** > 120 returns a column of boolean values that has **True** in place of those records in **flights.air_time** that are over 120, and **False** otherwise.

#### INSTRUCTIONS

1: Use the .filter() method to find all the flights that flew over 1000 miles two ways:

2: First, pass a SQL string to .filter() that checks whether the distance is greater than 1000. Save this as long_flights1.

3: Then pass a column of boolean values to .filter() that checks the same thing. Save this as long_flights2.

4: Use .show() to print heads of both DataFrames and make sure they're actually equal!

In [8]:
# Filter flights by passing a string
long_flights1 = flights.filter("distance > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.distance > 1000)

# Print the data to check they're equal
long_flights1.show()
long_flights2.show()

#print(long_flights2)
#print(long_flights1)

#flights.filter(flights.air_time > 120).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|              2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|               3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20| 2.566666666666667|
|2014|   11| 12|    2346|  

## 3:SELECTING

The Spark variant of SQL's **SELECT** is the **.select()** method. This method takes multiple arguments - one for each column you want to select. These arguments can either be the column name as a string (one for each column) or a column object (using the **df.colName** syntax). When you pass a column object, you can perform operations like addition or subtraction on the column to change the data contained in it, much like inside **.withColumn()**.

The difference between **.select()** and **.withColumn()** methods is that **.select()** returns only the columns you specify, while **.withColumn()** returns all the columns of the DataFrame in addition to the one you defined. It's often a good idea to drop columns you don't need at the beginning of an operation so that you're not dragging around extra data as you're wrangling. In this case, you would use **.select()** and not **.withColumn()**

### INSTRUCTIONS

1: Select the columns tailnum, origin, and dest from flights by passing the column names as strings. Save this as selected1.

2: Select the columns origin, dest, and carrier using the df.colName syntax and then filter the result using both of the filters already defined for you (filterA and filterB) to only keep flights from SEA to PDX. Save this as selected2.

In [10]:
# Select the first set of columns
selected1 = flights.select('tailnum', 'origin', 'dest')

# Select the second set of columns
temp = flights.select(flights.origin, flights.dest, flights.carrier)

# Define first filter
filterA = flights.origin == "SEA"

# Define second filter
filterB = flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)

selected2.show()

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 20 rows



## 4:SELECTING II

Similar to SQL, you can also use the **.select()** method to perform column-wise operations. When you're selecting a column using the **df.colName** notation, you can perform any column operation and the **.select()** method will return the transformed column. For example,

   **flights.select(flights.air_time/60)**
     
returns a column of flight durations in hours instead of minutes. You can also use the **.alias()** method to rename a column you're selecting. So if you wanted to **.select()** the column **duration_hrs** (which isn't in your DataFrame) you could do

   **flights.select((flights.air_time/60).alias("duration_hrs"))**

The equivalent Spark DataFrame method **.selectExpr()** takes SQL expressions as a string:
       
   **flights.selectExpr("air_time/60 as duration_hrs")**
       
with the SQL **as** keyword being equivalent to the .alias() method. To select multiple columns, you can pass multiple strings.

### INSTRUCTIONS
Create a table of the average speed of each flight both ways.

1: Calculate average speed by dividing the distance by the air_time (converted to hours). Use the .alias() method name this column "avg_speed". Save the output as the variable avg_speed.

2: Select the columns "origin", "dest", "tailnum", and avg_speed (without quotes!). Save this as speed1.

3: Create the same table using .selectExpr() and a string containing a SQL expression. Save this as speed2.

In [41]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed") 

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)
speed1.show()

# Create the same table using a SQL expression
speed2 = flights.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")
speed2.show()

#speed3 = flights.withColumn('avg_speed', flights.distance/(flights.air_time/60))

speed3 = flights.withColumn('avg_speed',avg_speed)
speed3.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

## 5:AGGREGATING

All of the common aggregation methods, like **.min()**, **.max()**, and **.count()** are GroupedData methods. These are created by calling the **.groupBy()** DataFrame method. You'll learn exactly what that means in a few exercises. For now, all you have to do to use these functions is call that method on your DataFrame. For example, to find the minimum value of a column, **col**, in a DataFrame, df, you could do
            
   **df.groupBy().min("col").show()
            
This creates a **GroupedData** object (so you can use the **.min()** method), then finds the minimum value in **col**, and returns it as a DataFrame.


### INSTRUCTIONS
1: Find the length of the shortest (in terms of distance) flight that left PDX by first .filter()ing and using the .min() method. Perform the filtering by referencing the column directly, not passing a SQL string.

2: Find the length of the longest (in terms of time) flight that left SEA by filter()ing and using the .max() method. Perform the filtering by referencing the column directly, not passing a SQL string.

In [11]:
flights = flights.withColumn("distance", flights.distance.cast('integer'))

# Find the shortest flight from PDX in terms of distance
flights.filter(flights.origin == "PDX").groupBy().min("distance").show()

flights = flights.withColumn("air_time", flights.air_time.cast('integer'))

# Find the longest flight from SEA in terms of air time
flights.filter(flights.origin == "SEA").groupBy().max('air_time').show()

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+

+-------------+
|max(air_time)|
+-------------+
|          409|
+-------------+



## 6:AGGREGATINGII

1: Use the .avg() method to get the average air time of Delta Airlines flights (where the carrier column has the value "DL") that left SEA. The place of departure is stored in the column origin. show() the result.

2: Use the .sum() method to get the total number of hours all planes in this dataset spent in the air by creating a column called duration_hrs from the column air_time. show() the result.

In [48]:
# Average duration of Delta flights
flights.filter(flights.carrier == "DL").filter(flights.origin == "SEA").groupBy().avg("air_time").show()

# Total hours in the air
flights.withColumn("duration_hrs", flights.air_time/60).groupBy().sum("duration_hrs").show()

+------------------+
|     avg(air_time)|
+------------------+
|188.20689655172413|
+------------------+

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



## 7:GROUPING AND AGGREGATING
Part of what makes aggregating so powerful is the addition of groups. PySpark has a whole class devoted to grouped data frames: **pyspark.sql.GroupedData**, which you saw in the last two exercises.

You've learned how to create a grouped DataFrame by calling the **.groupBy()** method on a DataFrame with no arguments.

Now you'll see that when you pass the name of one or more columns in your DataFrame to the **.groupBy()** method, the aggregation methods behave like when you use a **GROUP BY** statement in a SQL query!

### INSTRUCTIONS

1: Create a DataFrame called by_plane that is grouped by the column tailnum.

2: Use the .count() method with no arguments to count the number of flights each plane made.

3: Create a DataFrame called by_origin that is grouped by the column origin.

4: Find the .avg() of the air_time column to find average duration of flights from PDX and SEA

In [50]:
# Group by tailnum
by_plane = flights.groupBy("tailnum")

# Number of flights each plane made
by_plane.count().show()

# Group by origin
by_origin = flights.groupBy("origin")

# Average duration of flights from PDX and SEA
by_origin.avg("air_time").show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



## 8:GROUPING AND AGGREGATING II

In addition to the **GroupedData** methods you've already seen, there is also the **.agg()** method. This method lets you pass an aggregate column expression that uses any of the aggregate functions from the **pyspark.sql.functions** submodule.

This submodule contains many useful functions for computing things like standard deviations. All the aggregation functions in this submodule take the name of a column in a **GroupedData** table.

### INSTRUCTIONS

1: Import the submodule pyspark.sql.functions as F.

2: Create a GroupedData table called by_month_dest that's grouped by both the month and dest columns. Refer to the two columns by passing both strings as separate arguments.

3: Use the .avg() method on the by_month_dest DataFrame to get the average dep_delay in each month for each destination.

4: Find the standard deviation of dep_delay by using the .agg() method with the function F.stddev().

In [54]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

flights = flights.withColumn("dep_delay", flights.dep_delay.cast('integer'))
# Group by month and dest
by_month_dest = flights.groupBy("month", "dest")

# Average departure delay by month and destination
by_month_dest.avg("dep_delay").show()

# Standard deviation of departure delay
by_month_dest.agg(F.stddev("dep_delay")).show()

+-----+----+--------------------+
|month|dest|      avg(dep_delay)|
+-----+----+--------------------+
|   11| TUS| -2.3333333333333335|
|   11| ANC|   7.529411764705882|
|    1| BUR|               -1.45|
|    1| PDX| -5.6923076923076925|
|    6| SBA|                -2.5|
|    5| LAX|-0.15789473684210525|
|   10| DTW|                 2.6|
|    6| SIT|                -1.0|
|   10| DFW|  18.176470588235293|
|    3| FAI|                -2.2|
|   10| SEA|                -0.8|
|    2| TUS| -0.6666666666666666|
|   12| OGG|  25.181818181818183|
|    9| DFW|   4.066666666666666|
|    5| EWR|               14.25|
|    3| RDM|                -6.2|
|    8| DCA|                 2.6|
|    7| ATL|   4.675675675675675|
|    4| JFK| 0.07142857142857142|
|   10| SNA| -1.1333333333333333|
+-----+----+--------------------+
only showing top 20 rows

+-----+----+----------------------+
|month|dest|stddev_samp(dep_delay)|
+-----+----+----------------------+
|   11| TUS|    3.0550504633038935|
|   11| ANC|  

## JOINING

Another very common data operation is the join. A join will combine two different tables along a column that they share. This column is called the key. Examples of keys here include the tailnum and carrier columns from the flights table.

For example, suppose that you want to know more information about the plane that flew a flight than just the tail number. This information isn't in the flights table because the same plane flies many different flights over the course of two years, so including this information in every row would result in a lot of duplication. To avoid this, you'd have a second table that has only one row for each plane and whose columns list all the information about the plane, including its tail number. You could call this table planes

When you join the flights table to this table of airplane information, you're adding all the columns from the planes table to the flights table. To fill these columns with information, you'll look at the tail number from the flights table and find the matching one in the planes table, and then use that row to fill out all the new columns.

Now you'll have a much bigger table than before, but now every row has all information about the plane that flew that flight!

## JOINING II
In PySpark, joins are performed using the DataFrame method **.join()**. This method takes three arguments. The first is the second DataFrame that you want to join with the first one. The second argument, on, is the name of the key column(s) as a string. The names of the key column(s) must be the same in each table. The third argument, how, specifies the kind of join to perform. In this course we'll always use the value how="leftouter".

### INSTRUCTIONS

1: Examine the airports DataFrame by calling .show(). Note which key column will let you join airports to the flights table.

2: Rename the faa column in airports to dest by re-assigning the result of airports.withColumnRenamed("faa", "dest") to airports.

3: Join the flights with the airports DataFrame on the dest column by calling the .join() method on flights. Save the result as flights_with_airports.
  
  A: The first argument should be the other DataFrame, airports.
  
  B: The argument on should be the key column.
  
  C: The argument how should be "leftouter".

In [13]:
print(spark.catalog.listTables())
# Examine the data
airports = spark.read.csv('E:/STUDY/airports.csv', header=True)
print(airports.show())

# Rename the faa column
airports = airports.withColumnRenamed("faa", "dest")

# Join the DataFrames
flights_with_airports = flights.join(airports, 'dest', 'leftouter')

# Examine the new DataFrame
flights_with_airports.show()

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.78123

# Chapter3: GETTING STARTED WITH MACHINE LEARNING PIPELINES 

At the core of the **pyspark.ml** module are the **Transformer** and **Estimator** classes. Almost every other class in the module behaves similarly to these two basic classes.

**Transformer** classes have a **.transform()** method that takes a DataFrame and returns a new DataFrame; usually the original one with a new column appended. For example, you might use the class **Bucketizer** to create discrete bins from a continuous feature or the class **PCA** to reduce the dimensionality of your dataset using principal component analysis.

**Estimator** classes all implement a **.fit()** method. These methods also take a DataFrame, but instead of returning another DataFrame they return a model object. This can be something like a **StringIndexerModel** for including categorical data saved as strings in your models, or a **RandomForestModel** that uses the random forest algorithm for classification or regression.

In [15]:
file = "E:/STUDY/planes.csv"

# Read in the planes data
planes = spark.read.csv(file, header=True)

# Add planes_temp to the catalog
planes.createOrReplaceTempView('planes')

#Examine the tables in the catalog
print(spark.catalog.listTables())

planes.show()


[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='planes', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2

## EXERCISE

### 1: JOIN THE DATAFRAMES
1: First, rename the **year** column of **planes** to **plane_year** to avoid duplicate column names.

2: Create a new DataFrame called **model_data** by joining the **flights** table with planes using the tailnum column as the key.

In [16]:
# Rename year column
planes = planes.withColumnRenamed('year', 'plane_year')

# Join the DataFrames
model_data = flights.join(planes, on='tailnum', how="leftouter")
model_data.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|   NA|Turbo-fan|
| N559AS|2014|    1| 22|    1040|        5|    1505|        5|     AS|   851|   SEA| HNL

## DATA TYPES
Good work! Before you get started modeling, it's important to know that Spark only handles numeric data. That means all of the columns in your DataFrame must be either integers or decimals (called 'doubles' in Spark).

When we imported our data, we let Spark guess what kind of information each column held. Unfortunately, Spark doesn't always guess right and you can see that some of the columns in our DataFrame are strings containing numbers as opposed to actual numeric values.

To remedy this, you can use the **.cast()** method in combination with the **.withColumn()** method. It's important to note that **.cast()** works on columns, while **.withColumn()** works on DataFrames.

The only argument you need to pass to **.cast()** is the kind of value you want to create, in string form. For example, to create **integers**, you'll pass the argument **"integer"** and for decimal numbers you'll use **"double".

## 2:STRING TO INTEGER 

1: Use the method **.withColumn()** to **.cast()** the following columns to type **"integer"**. Access the columns using the **df.col** notation:
A: **model_data.arr_delay**
B: **model_data.air_time**
C: **model_data.month**
D: **model_data.plane_year**

In [28]:
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn("air_time", model_data.air_time.cast('integer'))
model_data = model_data.withColumn("month", model_data.month.cast('integer'))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast('integer'))

## 3:CREATE A NEW COLUMN

Create the column **plane_age** using the **.withColumn()** method and subtracting the year of manufacture (column **plane_year**) from the year (column **year**) of the flight.

In [29]:
model_data = model_data.withColumn("plane_age", model_data.year - model_data.plane_year)
model_data.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+-------+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|is_late|label|plane_age|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+-------+-----+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|   NA|Turbo-fan|  false|    0|   

## 4:MAKING A BOOLEAN

1: Use the **.withColumn()** method to create the column **is_late**. This column is equal to **model_data.arr_delay > 0**.

2: Convert this column to an integer column so that you can use it in your model and name it **label** (this is the default name for the response variable in Spark's machine learning routines).

3: Filter out missing values (this has been done for you).

In [30]:
# Create is_late
model_data = model_data.withColumn("is_late", model_data.arr_delay > 0)

# Convert to an integer
model_data = model_data.withColumn("label", model_data.is_late.cast('integer'))

# Remove missing values

model_data = model_data.filter("arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and plane_year is not NULL")
model_data.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+-------+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|is_late|label|plane_age|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+-------+-----+---------+
| N846VA|2014|   12|  8|     658|       -7|     935|       -5|     VX|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|   NA|Turbo-fan|  false|    0|   

## STRINGS AND FACTORS
As you know, Spark requires numeric data for modeling. So far this hasn't been an issue; even boolean columns can easily be converted to integers without any trouble. But you'll also be using the airline and the plane's destination as features in your model. These are coded as strings and there isn't any obvious way to convert them to a numeric data type.

Fortunately, PySpark has functions for handling this built into the **pyspark.ml.features**submodule. You can create what are called 'one-hot vectors' to represent the carrier and the destination of each flight. A one-hot vector is a way of representing a categorical feature where every observation has a vector in which all elements are zero except for at most one element, which has a value of one (1).

Each element in the vector corresponds to a level of the feature, so it's possible to tell what the right level is by seeing which element of the vector is equal to one (1).

The first step to encoding your categorical feature is to create a **StringIndexer**. Members of this class are **Estimator**s that take a DataFrame with a column of strings and map each unique string to a number. Then, the **Estimator** returns a **Transformer** that takes a DataFrame, attaches the mapping to it as metadata, and returns a new DataFrame with a numeric column corresponding to the string column.

The second step is to encode this numeric column as a one-hot vector using a **OneHotEncoder**. This works exactly the same way as the **StringIndexer** by creating an **Estimator** and then a **Transformer**. The end result is a column that encodes your categorical feature as a vector that's suitable for machine learning routines!

This may seem complicated, but don't worry! All you have to remember is that you need to create a **StringIndexer** and a **OneHotEncoder**, and the **Pipeline** will take care of the rest.

## 5:CARRIER

1: Create a **StringIndexer** called **carr_indexer** by calling **StringIndexer()** with **inputCol="carrier"** and **outputCol="carrier_index"**.

2: Create a **OneHotEncoder** called **carr_encoder** by calling **OneHotEncoder()** with **inputCol="carrier_index"** and **outputCol="carrier_fact"**.

### NOTE: 
The **inputCol** is the name of the column you want to index or encode, and the **outputCol** is the name of the new column that the **Transformer** should create.

In [31]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
# Create a StringIndexer
carr_indexer = StringIndexer(inputCol="carrier", outputCol="carrier_index")

# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol="carrier_index", outputCol="carrier_fact")

## 6: DESTINATION
Now you'll encode the **dest** column 

1: Create a **StringIndexer** called **dest_indexer** by calling **StringIndexer()** with **inputCol="dest"** and **outputCol="dest_index"**.

2: Create a **OneHotEncoder** called **dest_encoder** by calling **OneHotEncoder()** with **inputCol="dest_index"** and **outputCol="dest_fact"**.

In [32]:
# Create a StringIndexer
dest_indexer = StringIndexer(inputCol="dest", outputCol="dest_index")

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

## ASSEMBLE A VECTOR
The last step in the **Pipeline** is to combine all of the columns containing our features into a single column. This has to be done before modeling can take place because every Spark modeling routine expects the data to be in this form. You can do this by storing each of the values from a column as an entry in a vector. Then, from the model's point of view, every observation is a vector that contains all of the information about it and a label that tells the modeler what value that observation corresponds to.

Because of this, the **pyspark.ml.feature** submodule contains a class called **VectorAssembler**. This **Transformer** takes all of the columns you specify and combines them into a new vector column.

### INSTRUCTIONS
1: Create a **VectorAssembler** by calling **VectorAssembler()** with the **inputCols** names as a list and the **outputCol** name **"features"**.
The list of columns should be ["month", "air_time", "carrier_fact", "dest_fact", "plane_age"].

In [33]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol="features")

## CREATE THE PIPELINE
You're finally ready to create a **Pipeline**!

**Pipeline** is a class in the **pyspark.ml** module that combines all the **Estimators** and **Transformers** that you've already created. This lets you reuse the same modeling process over and over again by wrapping it up in one simple object. Neat, right?

### INSTRUCTORS
1: Import **Pipeline** from **pyspark.ml**.

2: Call the **Pipeline()** constructor with the keyword argument **stages** to create a **Pipeline** called **flights_pipe**.

3: **stages** should be a list holding all the stages you want your data to go through in the pipeline. Here this is just: [dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler]

In [34]:
# Import Pipeline
from pyspark.ml import Pipeline

# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

## TEST vs TRAIN
After you've cleaned your data and gotten it ready for modeling, one of the most important steps is to split the data into a test set and a train set. After that, don't touch your test data until you think you have a good model! As you're building models and forming hypotheses, you can test them on your training data to get an idea of their performance.

Once you've got your favorite model, you can see how well it predicts the new data in your test set. This never-before-seen data will give you a much more realistic idea of your model's performance in the real world when you're trying to predict or classify new data.

In Spark it's important to make sure you split the data after all the transformations. This is because operations like **StringIndexer** don't always produce the same index even when given the same list of strings.

### Why is it important to use a test set in model evaluation?
By evaluating your model with a test set you can get a good idea of performance on new data.


## TRANSFORM THE DATA
Hooray, now you're finally ready to pass your data through the **Pipeline** you created!

### INSTRUCTIONS
Create the DataFrame **piped_data** by calling the **Pipeline** methods **.fit()** and **.transform()** in a chain. Both of these methods take **model_data** as their only argument.

In [35]:
# Fit and transform the data
piped_data = flights_pipe.fit(model_data).transform(model_data)

## SPLIT THE DATA
Now that you've done all your manipulations, the last step before modeling is to split the data!

### INSTRUCTIONS
Use the DataFrame method **.randomSplit()** to split **piped_data** into two pieces, **training** with 60% of the data, and **test** with 40% of the data by passing the list [.6, .4] to the **.randomSplit()** method.


In [36]:
# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])

# Chapter4: MODEL TUNING AND SELECTION

## What is logistic regression?
The model you'll be fitting in this chapter is called a logistic regression. This model is very similar to a linear regression, but instead of predicting a numeric variable, it predicts the probability (between 0 and 1) of an event.

To use this as a classification algorithm, all you have to do is assign a cutoff point to these probabilities. If the predicted probability is above the cutoff point, you classify that observation as a 'yes' (in this case, the flight being late), if it's below, you classify it as a 'no'!

You'll tune this model by testing different values for several hyperparameters. A hyperparameter is just a value in the model that's not estimated from the data, but rather is supplied by the user to maximize performance. For this course it's not necessary to understand the mathematics behind all of these values - what's important is that you'll try out a few different choices and pick the best one.

### Why do you supply hyperparameters?
They improve model performance.

## CREATE THE MODELER
The **Estimator** you'll be using is a **LogisticRegression** from the **pyspark.ml.classification** submodule.

### INSTRUCTIONS
1: Import the **LogisticRegression** class from **pyspark.ml.classification**.

2: Create a **LogisticRegression** called **lr** by calling **LogisticRegression()** with no arguments.

In [37]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

## CROSS VALIDATION
In the next few exercises you'll be tuning your logistic regression model using a procedure called k-fold cross validation. This is a method of estimating the model's performance on unseen data (like your **test** DataFrame).

It works by splitting the training data into a few different partitions. The exact number is up to you, but in this course you'll be using PySpark's default value of three. Once the data is split up, one of the partitions is set aside, and the model is fit to the others. Then the error is measured against the held out partition. This is repeated for each of the partitions, so that every block of data is held out and used as a test set exactly once. Then the error on each of the partitions is averaged. This is called the cross validation error of the model, and is a good estimate of the actual error on the held out data.

You'll be using cross validation to choose the hyperparameters by creating a grid of the possible pairs of values for the two hyperparameters, **elasticNetParam** and **regParam**, and using the cross validation error to compare all the different models so you can choose the best one!

### What does cross validation allow you to estimate?
The model's error on held out data.

**NOTE:** The cross validation error is an estimate of the model's error on the test set.

## CREATE THE EVALUATOR
The first thing you need when doing cross validation for model selection is a way to compare different models. Luckily, the **pyspark.ml.evaluation** submodule has classes for evaluating different kinds of models. Your model is a binary classification model, so you'll be using the **BinaryClassificationEvaluator** from the **pyspark.ml.evaluation** module.

This evaluator calculates the area under the ROC. This is a metric that combines the two kinds of errors a binary classifier can make (false positives and false negatives) into a simple number. You'll learn more about this towards the end of the chapter!

### INSTRUCTIONS
1: Import the submodule **pyspark.ml.evaluation** as evals.

2: Create **evaluator** by calling **evals.BinaryClassificationEvaluator()** with the argument **metricName="areaUnderROC"**.

In [38]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

## MAKE A GRID
Next, you need to create a grid of values to search over when looking for the optimal hyperparameters. The submodule **pyspark.ml.tuning** includes a class called **ParamGridBuilder** that does just that (maybe you're starting to notice a pattern here; PySpark has a submodule for just about everything!).

You'll need to use the **.addGrid()** and **.build()** methods to create a grid that you can use for cross validation. The **.addGrid()** method takes a model parameter (an attribute of the model **Estimator**, **lr**, that you created a few exercises ago) and a list of values that you want to try. The **.build()** method takes no arguments, it just returns the grid that you'll use later.

### INSTRUCTIONS
1: Import the submodule **pyspark.ml.tuning** under the alias tune.

2: Call the class constructor **ParamGridBuilder()** with no arguments. Save this as **grid**.

3: Call the **.addGrid()** method on **grid** with **lr.regParam** as the first argument and **np.arange(0, .1, .01)** as the second argument. This second call is a function from the **numpy** module (imported as np) that creates a list of numbers from 0 to .1, incrementing by .01. Overwrite grid with the result.

3: Update **grid** again by calling the **.addGrid()** method a second time create a grid for **lr.elasticNetParam** that includes only the values [0, 1].

4: Call the **.build()** method on **grid** and overwrite it with the output.

In [39]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid = tune.ParamGridBuilder()

# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, .1, .01))
grid = grid.addGrid(lr.elasticNetParam, [0,1])

# Build the grid
grid = grid.build()

## MAKE THE VALIDATOR
The submodule **pyspark.ml.tuning** also has a class called **CrossValidator** for performing cross validation. This **Estimator** takes the modeler you want to fit, the grid of hyperparameters you created, and the evaluator you want to use to compare your models.

The submodule **pyspark.ml.tune** has already been imported as **tune**. You'll create the **CrossValidator** by passing it the logistic regression **Estimator** **lr**, the parameter **grid**, and the **evaluator** you created in the previous exercises.

### INSTRUCTIONS
1: Create a **CrossValidator** by calling **tune.CrossValidator()** with the arguments:

**estimator=lr**

**estimatorParamMaps=grid**

**evaluator=evaluator**

2: Name this object **cv**.

In [40]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid,
               evaluator=evaluator
               )

## FIT THE MODEL(s)
You're finally ready to fit the models and select the best one!

Unfortunately, cross validation is a very computationally intensive procedure. Fitting all the models would take too long on DataCamp.

To do this locally you would use the code:

#### Fit cross validation models
models = cv.fit(training)

#### Extract the best model
best_lr = models.bestModel

Remember, the training data is called **training** and you're using **lr** to fit a logistic regression model. Cross validation selected the parameter values **regParam=0** and **elasticNetParam=0** as being the best. These are the default values, so you don't need to do anything else with **lr** before fitting the model.

### INSTRUCTIONS
1: Create **best_lr** by calling **lr.fit()** on the training data.

2: Print **best_lr** to verify that it's an object of the **LogisticRegressionModel** class.

In [41]:
# Call lr.fit()
best_lr = lr.fit(training)

# Print best_lr
print(best_lr)

LogisticRegressionModel: uid = LogisticRegression_286799418c56, numClasses = 2, numFeatures = 81


## EVALUATING BINARY CLASSIFIERS
For this course we'll be using a common metric for binary classification algorithms call the AUC, or area under the curve. In this case, the curve is the ROC, or receiver operating curve. The details of what these things actually measure isn't important for this course. All you need to know is that for our purposes, the closer the AUC is to one (1), the better the model is!

## EVALUATE THE MODEL
Remember the test data that you set aside waaaaaay back in chapter 3? It's finally time to test your model on it! You can use the same evaluator you made to fit the model.

### INSTRUCTIONS
1: Use your model to generate predictions by applying **best_lr.transform()** to the **test** data. Save this as **test_results**.

2: Call **evaluator.evaluate()** on **test_results** to compute the AUC. Print the output.

In [43]:
# Use the model to predict the test set
test_results = best_lr.transform(test)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.6827307929101454
